In [2]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('lahman2016.sqlite')

In [ ]:
query = '''select * from Teams
inner join TeamsFranchises
on Teams.franchID == TeamsFranchises.franchID
where Teams.G >= 150 and TeamsFranchises.active == 'Y';
'''

In [ ]:
Teams = conn.execute(query).fetchall()

In [ ]:
teams_df = pd.DataFrame(Teams)
teams_df.head(5)

In [ ]:
cols = ['yearID','lgID','teamID','franchID','divID',
        'Rank','G','Ghome','W','L','DivWin','WCWin',
        'LgWin','WSWin','R','AB','H','2B','3B','HR',
        'BB','SO','SB','CS','HBP','SF','RA','ER','ERA',
        'CG','SHO','SV','IPouts','HA','HRA','BBA','SOA',
        'E','DP','FP','name','park','attendance','BPF',
        'PPF','teamIDBR','teamIDlahman45','teamIDretro',
        'franchID','franchName','active','NAassoc']
teams_df.columns = cols

In [ ]:
print(teams_df.head(2))

In [ ]:
len(teams_df)

In [ ]:
drop_cols = ['lgID','franchID','divID','Rank','Ghome',
             'L','DivWin','WCWin','LgWin','WSWin','SF',
             'name','park','attendance','BPF','PPF',
             'teamIDBR','teamIDlahman45','teamIDretro',
             'franchID','franchName','active','NAassoc']
df = teams_df.drop(drop_cols,axis=1)

In [ ]:
print(df.isnull().sum(axis=0).tolist())

In [ ]:
df = df.drop(['CS','HBP'],axis=1)

In [ ]:
df['SO'] = df['SO'].fillna(df['SO'].median())
df['DP'] = df['DP'].fillna(df['DP'].median())

In [ ]:
print(df.isnull().sum(axis=0).tolist())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(df['W'])
plt.xlabel('wins')
plt.title('histogram of wins')

In [ ]:
df['W'].mean()

In [ ]:
#create bins for each win values
def assign_win_bins(w):
    if w < 50:
        return 1
    if w >= 50 and w <= 69:
        return 2
    if w >= 70 and w <= 89:
        return 3
    if w >= 90 and w <= 109:
        return 4
    if w >= 110:
        return 5

In [ ]:
df['win_bins'] = df['W'].apply(assign_win_bins)

In [ ]:
plt.scatter(df['yearID'],df['W'],c=df['win_bins'])
plt.title('wins per year')
plt.xlabel('year')
plt.ylabel('wins')

In [ ]:
df = df[df['yearID']>1900]

In [ ]:
runs_per_year = {}
games_per_year = {}

for i, row in df.iterrows():
    year = row['yearID']
    runs = row['R']
    games = row['G']
    if year in runs_per_year:
        runs_per_year[year] = runs_per_year[year] + runs
        games_per_year[year] = games_per_year[year] + games
    else:
        runs_per_year[year] = runs
        games_per_year[year] = games

In [ ]:
print(runs_per_year)

In [ ]:
print(games_per_year)

In [ ]:
mlb_runs_per_game = {}

for k,v in games_per_year.items():
    year = k
    games = v
    runs = runs_per_year[year]
    mlb_runs_per_game[year] = runs/games

In [ ]:
print(mlb_runs_per_game)

In [ ]:
lists = sorted(mlb_runs_per_game.items())
x,y = zip(*lists)

In [ ]:
plt.plot(x,y)
plt.title('mlb yearly runs per game')
plt.xlabel('year')
plt.ylabel('mlb runs per game')

In [ ]:
def assign_label(year):
    if year < 1920:
        return 1
    elif year >= 1920 and year <= 1941:
        return 2
    elif year >= 1942 and year <= 1945:
        return 3
    elif year >= 1946 and year <= 1962:
        return 4
    elif year >= 1963 and year <= 1976:
        return 5
    elif year >= 1977 and year <= 1992:
        return 6
    elif year >= 1993 and year <= 2009:
        return 7
    elif year >= 2010:
        return 8

In [ ]:
df['year_label'] = df['yearID'].apply(assign_label)

In [ ]:
dummy_df = pd.get_dummies(df['year_label'],prefix='era')

In [ ]:
dummy_df.head()

In [ ]:
df = pd.concat([df,dummy_df],axis=1)

In [ ]:
df.head(2)

In [ ]:
def assign_mlb_rpg(year):
    return mlb_runs_per_game[year]
df['mlb_rpg'] = df['yearID'].apply(assign_mlb_rpg)

In [ ]:
def assign_decade(year):
    if year < 1920:
        return 1910
    elif year >= 1920 and year <= 1929:
        return 1920
    elif year >= 1930 and year <= 1939:
        return 1930
    elif year >= 1940 and year <= 1949:
        return 1940
    elif year >= 1950 and year <= 1959:
        return 1950
    elif year >= 1960 and year <= 1969:
        return 1960
    elif year >= 1970 and year <= 1979:
        return 1970
    elif year >= 1980 and year <= 1989:
        return 1980
    elif year >= 1990 and year <= 1999:
        return 1990
    elif year >= 2000 and year <= 2009:
        return 2000
    elif year >= 2010:
        return 2010

In [ ]:
df['decade_label'] = df['yearID'].apply(assign_decade)

In [ ]:
decade_df = pd.get_dummies(df['decade_label'],prefix='decade')
decade_df.head()

In [ ]:
df = pd.concat([df,decade_df],axis=1)

In [ ]:
df = df.drop(['yearID','year_label','decade_label'],axis=1)

In [ ]:
df['R_per_game'] = df['R']/df['G']
df['RA_per_game'] = df['RA']/df['G']

In [ ]:
fig = plt.figure(figsize=(12,6))

ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

ax1.scatter(df['R_per_game'],df['W'],c='blue')
ax1.set_title('runs per game vs wins')
ax1.set_ylabel('wins')
ax1.set_xlabel('runs per game')

ax2.scatter(df['RA_per_game'],df['W'],c='red')
ax2.set_title('runs allowed per game vs wins')
ax2.set_xlabel('runs allowed per game')

In [ ]:
df.corr()['W']

In [ ]:
attributes = ['G','R','AB','H','2B','3B','HR','BB','SO','SB','RA','ER','ERA','CG',
'SHO','SV','IPouts','HA','HRA','BBA','SOA','E','DP','FP','era_1','era_2','era_3','era_4','era_5','era_6','era_7','era_8','decade_1910','decade_1920','de

In [ ]:
data_attributes = df[attributes]

In [ ]:
df.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
score_dict = {}
for i in range(2,11):
    km = KMeans(n_clusters = i, random_state=1)
    l = km.fit_predict(data_attributes)
    score = metrics.silhouette_score(data_attributes,l)
    score_dict[i] = score

In [ ]:
score_dict

In [ ]:
kmeans_model = KMeans(n_clusters = 6, random_state=1)

In [ ]:
#determine euclidian distances for each data points
distances = kmeans_model.fit_transform(data_attributes)

In [ ]:
distances

In [ ]:
#create scatter plot colored by kmeans clusters
labels = kmeans_model.labels_
plt.scatter(distances[:,0],distances[:,1],c=labels)
plt.title('kmeans clusters')

In [ ]:
df['labels'] = labels
attributes.append('labels')
df.head()

In [ ]:
numeric_cols = ['G','R','AB','H','2B','3B','HR','BB','SO','SB','RA','ER','ERA','CG','SHO','SV','IPouts','HA','HRA','BBA','SOA','E','DP','FP','era_1','

In [ ]:
data = df[numeric_cols]

In [ ]:
data.head()

In [ ]:
train = data.sample(frac = 0.75, random_state=1)
test = data.loc[~data.index.isin(train.index)]

In [ ]:
x_train = train[attributes]
y_train = train['W']
x_test = test[attributes]
y_test = test['W']

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
lr = LinearRegression(normalize=True)

In [ ]:
lr.fit(x_train,y_train)

In [ ]:
predictions = lr.predict(x_test)

In [ ]:
mae = mean_absolute_error(y_test,predictions)

In [ ]:
mae

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
rrm = RidgeCV(alphas=(0.01,0.1,1,10),normalize=True)

In [ ]:
rrm.fit(x_train,y_train)

In [ ]:
predictions = rrm.predict(x_test)

In [ ]:
mae_rrm = mean_absolute_error(y_test,predictions)
mae_rrm